# Tutorial: Retrieval of documents from a corpus using Neural Information Retrieval (IR)

In this tutorial you'll learn how to use a popular Neural IR system called ColBERT [Khattab and Zaharia](https://arxiv.org/abs/2004.12832).


## Preparing a Colab Environment to run this tutorial ##

Make sure to "Enable GPU Runtime" -> make a URL with a page with screenshots on how to do this.

## Installing PrimeQA

First, we need to include the required modules.


In [ ]:
%%bash

pip install --upgrade pip
pip install primeqa

## Pre-process your document collection here to be ready to be stored in your Neural Search Index.
In this step we download a publicly available .csv file from a Google Drive location and save it as .tsv.

In [ ]:
# save your input document as a .tsv
import pandas as pd
url='https://drive.google.com/file/d/1LULJRPgN_hfuI2kG-wH4FUwXCCdDh9zh/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.to_csv('input.tsv', sep='\t')

## Step 1: Init -- Initialize your model. In PrimeQA for searching through your corpus, we use a class called SearchableCorpus.

For ColBERT, you need to point to an encoder available via the HuggingFace model hub.

In [ ]:
from primeqa.components.retriever.searchable_corpus import SearchableCorpus
collection = SearchableCorpus(model_name="PrimeQA/DrDecr_XOR-TyDi_whitebox-2", 
                              batch_size=64, top_k=10, retriever='colbert')

## Step 2: Add your documents into the searchable corpus through PrimeQA's built-in pre-processor.

PrimeQA has a built-in class called DocumentCollection which pre-processes input.tsv to match the following format as needed by ColBERT:

`id \t text \t title_of_document`

Note: since ColBERT is based on an encoder language model the typical sequence length is 512 max sub-word tokens. So please make sure your documents are split into text length of ~220 words.

In [ ]:
from primeqa.ir.util.corpus_reader import DocumentCollection
doc_collection = DocumentCollection("input.tsv")

collection.add_documents(doc_collection.get_processed_collection())

## Step 3: Search -- start asking questions.

Your queries can be a list. You can also retrieve the scores of retrieved documents.

In [ ]:
queries = ['When was Idaho split in two?' , 'Who was Danny Nozel']
retrieved_doc_ids, passages = collection.search(queries)
import json
print(json.dumps(passages, indent = 4))